## E9. Movielens 영화 추천 실습
MovieLens 에서 제공하는 1M Dataset 사용해서 영화추천 모델을 만들어 보는 프로젝트다. 다양한 유저들의 영화 별점을 바탕으로, 유저의 영화 취향을 파악하고 비슷한 영화를 추천하는 모델이다.     

사용자들은 영화의 별점을 1부터 5까지 부여했다. 보통 별점은 explicit data 이지만 우리는 이를 implicit data라고 간주하고 평가하고 했다. 별점을 시청횟수로 해석했다. 또한 유저가 3점 미만으로 준 평가는 영화를 선호하지 않는다고 가정하고 데이터에서 제외했다. 

### Executive Summary 

모델의 사양은 다음과 같다. 
* 모델명: AlternatingLeastSquares
* factors:1000
* regularization:0.01
* iterations: 50

내가 선택한 영화 'sixth sense'의 모델 평가 결과는 0.98. 1에 가깝게 나왔다.      
랜덤으로 선택한 영화 -0.014로 매우 낮았다. 

* 식스센스와 비슷한 영화     
Sixth Sense, The (1999) 1.0000001        
Heaven's Burning (1997) 0.32267874 - 스릴러 영화라는점이 식스센스와 비슷하다.       
Project Moon Base (1953) 0.3213943       
Match, The (1999) 0.31924695       
I Got the Hook Up (1998) 0.31874764                                     


* 추천영화         
Omen, The (1976) 0.17831232 - 오멘을 재밌게 보긴 했다.    
Cat's Eye (1985) 0.14193161 - 스티븐킹 소설을 원작으로 했다고 하는데. 스티븐킹 소설 좋아하는데 취향을 잘 파악한것 같다.     
Flirting With Disaster (1996) 0.12639093    
Event Horizon (1997) 0.12438591    
Final Destination (2000) 0.12087625    

### Table of Contents 
1. 데이터 가져오기, 이해하기
2. 데이터 전처리 
3. 데이터 분석 
4. 내가 선호하는 영화 5가지 골라서 rating에 추가해주기
5. user_id와 movie_id 인덱스화 시키기 
6. csr matrix 만들기 
7. 모델 훈련 (als_model = AlternatingLeastSquares)
8. 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.
9. 내가 좋아하는 영화와 비슷한 영화추천받자. 
10. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

###  필요한 모듈 가져오기

In [68]:
import os
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares


### 데이터 가져오기,  이해하기

README.txt 에 따르면 데이터는 아래와 같다. 
- UserIDs range between 1 and 6040 
- MovieIDs range between 1 and 3952
- Ratings are made on a 5-star scale (whole-star ratings only)
- Timestamp is represented in seconds since the epoch as returned by time(2)
- Each user has at least 20 ratings

데이터 분석할 때는 별점을 시청횟수로 해석해서 생각해 주기로 한다. 

In [69]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
original_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [70]:
ratings = pd.DataFrame(ratings)

### 데이터 전처리 

1. 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 데이터셋에서 제외
2. rating 컬럼의 이름을 count로 바꿉니다.
3. ratings를 분석했을때 영화제목을 movies 파일에서 꺼내오기 위해 id to title의 dictionary 형태로 만들어준다.  
4. rating에서 timestamp 는 필요없으므로 없애주기로 한다.   
5. 결측치 확인 


In [71]:
# rating 3이상만 포함시키기 
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {original_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / original_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [72]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [73]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')

In [74]:
id_to_title = dict(zip(movies['movie_id'], movies['title']))

In [75]:
ratings = ratings.drop(columns=['timestamp'])
ratings.tail()

,user_id,movie_id,count
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4


In [76]:
ratings.isnull().describe()

,user_id,movie_id,count
count,836478,836478,836478
unique,1,1,1
top,False,False,False
freq,836478,836478,836478


In [77]:
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()
print(len(user_unique))
print(len(movie_unique))

6039
3628


### 데이터 분석 

* ratings에 있는 유니크한 영화 개수
* rating에 있는 유니크한 사용자 수
* 가장 인기 있는 영화 30개(인기순)

In [78]:
ratings['movie_id'].nunique()

3628

In [79]:
ratings['user_id'].nunique()

6039

In [80]:
# 사람들이 많이 본 영화 top30 
watch_count = ratings.groupby('movie_id')['user_id'].count()
watch_count.sort_values(ascending=False, inplace=True)

In [81]:
watch_count.map(id_to_title.get).head(30)

movie_id
2858                            Cry in the Dark, A (1988)
260                                      Ennui, L' (1998)
1196                                     Guinevere (1999)
1210                                  Ghostbusters (1984)
2028                                    True Crime (1999)
589                              Eight Days a Week (1997)
593                            My Favorite Martian (1999)
1198                                      Soul Man (1986)
1270                Texas Chainsaw Massacre 2, The (1986)
2571                             Down in the Delta (1998)
480                                           Clue (1985)
2762                                    Home Fries (1998)
608                                         Fletch (1985)
110                                        Beloved (1998)
1580                          What Dreams May Come (1998)
527                                No Small Affair (1984)
1197                                          Hero (1992)
2396 

### 내가 선호하는 영화 5가지 골라서 rating에 추가해주기

In [82]:
s_movies = movies.loc[movies['title'].str.contains('truman', case=False)]
s_movies

,movie_id,title,genre
1636,1682,"Truman Show, The (1998)",Drama


In [83]:
# forest gump, sixth sense, shawshank redemption, american beauty, truman show
fav_movies_id = [356, 2762, 318, 2858, 1682]
fav_movies_rating = [5, 5, 5, 5, 5]
my_list = pd.DataFrame({'user_id':[6041]*5, 'movie_id': fav_movies_id, 'count': fav_movies_rating})

In [84]:
ratings = ratings.append(my_list)

In [85]:
ratings = ratings.reset_index(drop=True)

### user_id와 movie_id 인덱스화 시키기 

In [86]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}


In [87]:
temp_user_id = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_id) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_id   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

temp_movie_id = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_id) == len(ratings):
    print('movie column indexing OK!!')
    ratings['movie_id'] = temp_movie_id
else:
    print('movie column indexing Fail!!')

user_id column indexing OK!!
movie column indexing OK!!


### CSR matrix 만들기

In [88]:
num_user = ratings['user_id'].nunique()   # including my addition
num_movies = ratings['movie_id'].nunique()
print(num_user)
print(num_movies)

6040
3628


In [89]:
csr_data = csr_matrix((ratings['count'], (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movies))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

### 모델 훈련 (als_model = AlternatingLeastSquares)

In [90]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [91]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=1000, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

In [92]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [93]:
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

### 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

In [104]:
# user index, 식스센스 index 
print(user_to_idx[6041])
print(movie_to_idx[2762])

6039
38


In [105]:
# 내가 선택한 영화 (식스센스)
user_vector, movie_vector = als_model.user_factors[6039], als_model.item_factors[38]
np.dot(user_vector, movie_vector)

0.9838213

In [106]:
# 다른 영화 (랜덤한 영화)
another_vector = als_model.item_factors[3500] 
np.dot(user_vector, another_vector)

-0.01418808

### 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [107]:
fav_movie = 'sixth sense'
similar_movies = als_model.similar_items(38, N=5)
similar_movies

[(38, 1.0000001),
 (3591, 0.32267874),
 (3534, 0.3213943),
 (3600, 0.31924695),
 (3523, 0.31874764)]

In [98]:
idx_to_movie = {i:m for m, i in movie_to_idx.items()}

In [99]:
def get_title(idx):
    movie_id = idx_to_movie[idx]
    title = id_to_title[movie_id]
    return title 

In [108]:
for item in similar_movies: 
    movie_id, rating = item 
    title = get_title(movie_id)
    print (title, rating)

Sixth Sense, The (1999) 1.0000001
Heaven's Burning (1997) 0.32267874
Project Moon Base (1953) 0.3213943
Match, The (1999) 0.31924695
I Got the Hook Up (1998) 0.31874764


### 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [101]:
user = 6039 # user_id index
movie_recommended = als_model.recommend(user, csr_data, N=5, filter_already_liked_items=True)
movie_recommended

[(712, 0.051231954),
 (1646, 0.047962505),
 (1448, 0.0465714),
 (374, 0.046292283),
 (184, 0.044748887)]

In [102]:
for item in movie_recommended: 
    movie_id, rating = item 
    title = get_title(movie_id)
    print (title, rating)

To Wong Foo, Thanks for Everything! Julie Newmar (1995) 0.051231954
Yellow Submarine (1968) 0.047962505
Next Stop, Wonderland (1998) 0.0465714
Exorcist, The (1973) 0.046292283
Crocodile Dundee (1986) 0.044748887


In [103]:
fav_movies_id = [356, 2762, 318, 2858, 1682]
title_all = []

for item in fav_movies_id:
    title = id_to_title[item]
    title_all.append(title)

print(title_all)


['Forrest Gump (1994)', 'Sixth Sense, The (1999)', 'Shawshank Redemption, The (1994)', 'American Beauty (1999)', 'Truman Show, The (1998)']
